# Fashion MNIST Deep Learning Project

## 1. Problem Definition
## 2. Load Data
## 3. Train/Validation/Test Split
## 4. Preprocessing (Split first, then normalize)
## 5. Build CNN Model
## 6. Train With Early Stopping
## 7. Evaluate on Test
## 8. Conclusions

In [1]:
#Imports and Reproducibility

import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow:", tf.__version__)


TensorFlow: 2.19.0


In [3]:
#Load Dataset

from tensorflow.keras.datasets import fashion_mnist

(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
print("Train full:", X_train_full.shape, y_train_full.shape)
print("Test      :", X_test.shape, y_test.shape)


Train full: (60000, 28, 28) (60000,)
Test      : (10000, 28, 28) (10000,)


In [4]:
#Split Train/Validation

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full,
    test_size=0.15,
    random_state=42,
    stratify=y_train_full
)

print("Train:", X_train.shape, y_train.shape)
print("Val  :", X_val.shape, y_val.shape)
print("Test :", X_test.shape, y_test.shape)


Train: (51000, 28, 28) (51000,)
Val  : (9000, 28, 28) (9000,)
Test : (10000, 28, 28) (10000,)


In [5]:
#Normalizing after Split & Channel Dimension

X_train = X_train.astype("float32") / 255.0
X_val   = X_val.astype("float32") / 255.0
X_test  = X_test.astype("float32") / 255.0

# CNN expects (N, H, W, C)
X_train = X_train[..., np.newaxis]
X_val   = X_val[..., np.newaxis]
X_test  = X_test[..., np.newaxis]

print(X_train.shape, X_val.shape, X_test.shape)


(51000, 28, 28, 1) (9000, 28, 28, 1) (10000, 28, 28, 1)


In [6]:
#Building a Simple CNN

from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Input(shape=(28, 28, 1)),

    layers.Conv2D(32, kernel_size=3, activation="relu"),
    layers.MaxPooling2D(),

    layers.Conv2D(64, kernel_size=3, activation="relu"),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),

    layers.Dense(10, activation="softmax")
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
#Train with early stopping

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=3,
        restore_best_weights=True
    )
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=128,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 38s 91ms/step - accuracy: 0.6760 - loss: 0.9102 - val_accuracy: 0.8587 - val_loss: 0.3878
Epoch 2/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 36s 90ms/step - accuracy: 0.8529 - loss: 0.4153 - val_accuracy: 0.8798 - val_loss: 0.3240
Epoch 3/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 42s 94ms/step - accuracy: 0.8742 - loss: 0.3493 - val_accuracy: 0.8868 - val_loss: 0.3034
Epoch 4/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 36s 91ms/step - accuracy: 0.8867 - loss: 0.3124 - val_accuracy: 0.8936 - val_loss: 0.2827
Epoch 5/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 42s 93ms/step - accuracy: 0.8919 - loss: 0.2933 - val_accuracy: 0.8982 - val_loss: 0.2709
Epoch 6/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 40s 90ms/step - accuracy: 0.9010 - loss: 0.2692 - val_accuracy: 0.9030 - val_loss: 0.2660
Epoch 7/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 37s 93ms/step - accuracy: 0.9084 - loss: 0.2521 - val_accuracy: 0.9067 - val_loss: 0.2570
Epoch 8/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 37s 93ms/step - accuracy: 0.9132 - loss: 0.2392 - 

In [8]:
# Test Evaluation

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print("Test accuracy:", test_acc)
print("Test loss:", test_loss)


Test accuracy: 0.9120000004768372
Test loss: 0.24534103274345398


#Conclusion

A simple CNN achieved ~91–92% validation accuracy on Fashion-MNIST with early stopping. Training and validation curves showed mild overfitting but good generalization, and test performance was consistent with validation, indicating a stable model